In [ ]:
from googleapiclient.discovery import build
from dotenv import load_dotenv
import os
import pandas as pd
from IPython.display import JSON
import numpy as np
import isodate
from datetime import datetime

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwid", None)

GETTING DATA IN YOUTUBE

In [ ]:
load_dotenv()
api_key = os.getenv("YOUTUBE_API_KEY")

In [ ]:
channel_ids = ['UCqaQzGvnFzaEPFx-f326pog', 
               #More channels here
              ]

In [ ]:
 # Get credentials and create an API client
youtube = build(
    'youtube', 'v3', developerKey=api_key)

In [2]:
def get_channel_stats(youtube, channel_ids):

    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id = ','.join(channel_ids)
    )
    response = request.execute()

    # loop through items:
    for item in response['items']:
        data = {'Channel Name': item['snippet']['title'],
                'Subscribers': item['statistics']['subscriberCount'],
                'Views': item['statistics']['viewCount'],
                'Total Video': item['statistics']['videoCount'],
                'Playlist ID': item['contentDetails']['relatedPlaylists']['uploads']     
        }
        all_data.append(data)
    return(pd.DataFrame(all_data))

In [ ]:
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

GETTING VIDEO ID

In [ ]:
playlist_id = 'UUqaQzGvnFzaEPFx-f326pog'

def get_video_ids(youtube, playlist_id):

    request = youtube.playlistItems().list(
                part="contentDetails",
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
    
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part="contentDetails",
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
        
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
            next_page_token = response.get('nextPageToken')
    
    
    return video_ids


In [ ]:
video_ids = get_video_ids(youtube, playlist_id)

In [ ]:
len(video_ids)

GETTING EACH VIDEO'S DETAILS

In [ ]:
def get_video_details(youtube, video_ids):  

    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
                part="snippet,contentDetails,statistics", 
                id = ','.join(video_ids[i:i+50])
            )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {'snippet':['title', 'tags', 'publishedAt'],
                            'statistics': ['viewCount', 'likeCount' , 'commentCount'],
                            'contentDetails':['duration', 'caption']
                            }
            video_info = {}
            video_info['video_ids'] = video['id']
            
            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
            
            all_video_info.append(video_info)
        
    return pd.DataFrame(all_video_info)


In [ ]:
video_df = get_video_details(youtube, video_ids)


CLEANING DATA

In [ ]:
video_df.drop_duplicates(subset = 'video_ids',inplace=True)

In [ ]:
video_df.drop(columns=['video_ids', 'caption'], inplace=True)

In [ ]:
def converting_iso_to_normal(x):
    datetime_obj = datetime.fromisoformat(x)
    return datetime_obj.strftime('%Y-%m-%d %H:%M:%S %a')

In [ ]:
video_df['publishedAt'] = video_df['publishedAt'].apply(converting_iso_to_normal)

In [ ]:
def converting_duration(x):
    duration = isodate.parse_duration(x)
    return duration.total_seconds()

In [ ]:
video_df['duration'] = video_df['duration'].apply(converting_duration)

In [ ]:
video_df.dtypes

In [ ]:
numeric_columns = ['viewCount', 'likeCount', 'commentCount']

In [ ]:
video_df[numeric_columns] = video_df[numeric_columns].apply(pd.to_numeric, axis=1, errors = 'coerce')

In [ ]:
video_df.drop(columns='tags', inplace=True)

In [ ]:
video_df[['Date','Time', 'Weekday']] = video_df['publishedAt'].str.split(n = 3, expand=True)

In [ ]:
video_df.drop(columns='publishedAt', inplace=True)

In [ ]:
video_df.drop(columns=['Time', 'Weekday'], inplace=True)

CREATING DATAFRAMES FOR EACH VIDEO TYPES

CREATING TEU TRY DATAFRAME


In [ ]:
teu_try_df = video_df.loc[video_df['title'].str.contains('TẾU TRY')]

In [ ]:
teu_try_df.to_excel('D:/teu_try_df.xlsx', index=False)

CREATING SHORTS DATAFRAME


In [ ]:
short_df = video_df.loc[video_df['title'].str.contains('short')]

In [ ]:
short_df.to_excel('D:/short.xlsx', index = False)

CREATING DAU TEU DATAFRAME


In [ ]:
dau_teu_df = video_df.loc[video_df['title'].str.contains('ĐẤU TẾU')]

In [ ]:
dau_teu_df.to_excel('D:/dau_teu.xlsx', index=False)

CREATING UNG TAC DATAFRAME

In [ ]:
ung_tac_df = video_df.loc[video_df['title'].str.contains('TẾU ỨNG TÁC')]

In [ ]:
ung_tac_df.to_excel('D:/ung_tac.xlsx', index = False )

CREATING SPECIAL DATAFRAME

In [ ]:
special_df = video_df.loc[video_df['title'].str.contains('SPECIAL') | video_df['title'].str.contains('TẾU QUÂN')]

CLEANING SPECIAL DATAFRAME

In [ ]:
special_df = special_df[~special_df['title'].str.contains('#[1-5]', regex=True, na=False)]

ADDING COMEDIANS TO EACH VIDEO

In [ ]:
special_df["Comedian"] = ''

In [ ]:
special_df.loc[79, 'Comedian'] = 'Nhi Võ, Minh Ti, Phương Nam, Trọng Phan, Uy Lê'
special_df.loc[99, 'Comedian'] = 'Trọng Phan, Trung Coffee, Phương Nam, Minh Ti'     
special_df.loc[136, 'Comedian'] = 'Nhi Võ, Trung Coffee, Trọng Phan, Phương Nam'     
special_df.loc[163, 'Comedian'] = 'Trung Coffee, Minh Ti, Tùng BT, Trọng Phan, Phương Nam'     
special_df.loc[220, 'Comedian'] = 'Phương Nam, Trọng Phan, Trung Coffee, Minh Ti, Uy Lê'     
special_df.loc[234, 'Comedian'] = 'Minh Ti, Trung Coffee, Thanh, Uy Nguyễn, Trọng Phan, Tùng BT, Anh Khôi, Phương Nam, Chaiyo Thương, Uy Lê'
special_df.loc[244, 'Comedian'] = 'Uy Lê, Anh Khôi, Tùng BT, Phương Nam'     
special_df.loc[245, 'Comedian'] = 'Minh Ti, Chaiyo Thương, Uy Nguyễn, Trọng Phan'     
special_df.loc[254, 'Comedian'] = 'Trọng Phan, Anh Khôi, Phương Nam, Uy Lê'     
special_df.loc[257, 'Comedian'] = 'Uy Lê, Uy Nguyễn, Phương Nam, Chaiyo Thương, Uy Lê, Minh Ti'     
special_df.loc[260, 'Comedian'] = 'Trọng Phan, Uy Lê, Minh Ti'     
special_df.loc[266, 'Comedian'] = 'Trọng Phan, Anh Khôi, Minh Ti, Thanh, Phương Nam'     
special_df.loc[272, 'Comedian'] = 'Uy Lê, Tùng BT, Minh Ti, Trọng Phan, Phương Nam'                                         

In [ ]:
special_df.rename(columns={'viewCount' : 'View'}, inplace = True)
special_df.rename(columns={'likeCount' : 'Like'}, inplace = True)
special_df.rename(columns={'commentCount' : 'Comment'}, inplace = True)
special_df.rename(columns={'duration' : 'Duration'}, inplace = True)
special_df.rename(columns={'title' : 'Title'}, inplace = True)

CREATING MODIFIED SPECIAL DATAFRAME

In [ ]:
modified_data = []

for index, row in special_df.iterrows():
    comedians = row['Comedian'].split(',')
    
    for comedian in comedians:
        new_row = {
            'Title': row['Title'],
            'View': row['View'],
            'Like': row['Like'],
            'Comment': row['Comment'],
            'Duration': row['Duration'],
            'Date': row['Date'],
            'Comedian': comedian.strip()
        }
        modified_data.append(new_row)
modified_df = pd.DataFrame(modified_data)

In [ ]:
modified_df.to_excel('D:/modified_special.xlsx', index=False)

In [ ]:
special_df.to_excel("D:/SaiGon_Teu_Special.xlsx", index = False)

In [ ]:
video_df.to_excel("D:/All_SaiGon_Teu.xlsx", index = False)

CREATING HAI DOC THOAI DATAFRAME


In [ ]:
df1 = video_df.loc[video_df['title'].str.lower().str.contains('hài độc thoại')]

CLEANING HAI DOC THOAI DATAFRAME


In [ ]:
special = df1['title'].str.contains('SPECIAL')
df1.drop(df1[special].index, inplace = True)
short = df1['title'].str.contains('#Shorts')
df1.drop(df1[short].index, inplace = True)         

In [ ]:
df1.drop([153, 156, 158, 160, 181, 251, 281], inplace = True)

In [ ]:
df1.loc[df1['title'].str.contains('Khách Mời'), df1.columns]

In [ ]:
for index, row in df1.iterrows():
    if 'Khách Mời Saigon Tếu' in row['title']:
        df1.at[index, 'title'] = row['title'].replace('HÀI ĐỘC THOẠI - ', '')
        df1.at[index, 'title'] = df1.at[index, 'title'].replace(' - Khách Mời Saigon Tếu', '')

In [ ]:
for index, row in df1.iterrows():
    if '| Hài Độc Thoại' in row['title']:                
        df1.at[index, 'title'] = row['title'].replace(' | Hài Độc Thoại Saigon Tếu', '')
        df1.at[index, 'title'] = df1.at[index, 'title'].replace(' Saigon Tếu | Hài Độc Thoại', '')

In [ ]:
df1.loc[29 , 'title'] = 'Happy Bít Tết - Phương Nam'
df1.loc[16 , 'title'] = 'Thời Trang Câu Hỏi - Huỳn Khanh'

In [ ]:
for index, row in df1.iterrows():
    if 'HÀI ĐỘC THOẠI' in row['title']:
        df1.at[index, 'title'] = row['title'].replace('HÀI ĐỘC THOẠI - ', '')
        df1.at[index, 'title'] = df1.at[index, 'title'].replace(' Saigon Tếu', '')
        df1.at[index, 'title'] = df1.at[index, 'title'].replace('HÀI ĐỘC THOẠI PURGATORY - ', '')

In [ ]:
def formating(x):
    if x.count('-') >= 2:
        last_index = x.rfind('-')
        x = x.replace('-', ' ', x.count('-'))
        x = x[:last_index] + '-' + x[last_index+1:]
    return x
df1['title'] = df1['title'].apply(formating)



In [ ]:
df1[['Title', 'Comedian']] = df1['title'].str.split('-', n = 2, expand = True)

In [ ]:
df1.drop(columns = 'title', inplace = True)    

In [ ]:
df1.loc[df1['Comedian'] == '', df1.columns]
df1.loc[140, 'Comedian'] = 'Uy Lê'
df1.loc[205, 'Comedian'] = 'Uy Lê'
df1.loc[291, 'Comedian'] = 'Tùng BT'
df1.loc[df1['Comedian'] == '', df1.columns]

In [ ]:
df1.drop(columns = 'title', inplace = True)

In [ ]:
df1.rename(columns={'viewCount' : 'View'}, inplace = True)
df1.rename(columns={'likeCount' : 'Like'}, inplace = True)
df1.rename(columns={'commentCount' : 'Comment'}, inplace = True)
df1.rename(columns={'duration' : 'Duration'}, inplace = True)


In [ ]:
df1.loc[df1['Comedian'].str.contains('Quốc Khánh'), df1.columns]

In [ ]:
df1 = df1[['Title', 'Comedian', 'viewCount', 'likeCount', 'commentCount', 'duration', 'Date']]

In [ ]:
df1['Comedian'] = df1['Comedian'].astype(str)


In [ ]:
df1['Comedian'] = df1['Comedian'].str.title()

In [ ]:
def remove_space(x):
    space_count = x.count(' ')
    if space_count >= 2:
        x = x.strip()
    return x

In [ ]:
df1['Comedian'] = df1['Comedian'].apply(remove_space)

In [ ]:
df1['Title'] = video_df['title']

In [ ]:
df1

In [ ]:
df1.to_excel("D:/SaiGon_Teu.xlsx", index = False)